In [1]:
import numpy as np
import os   # 운영체제와 상호작용하기 위한 모듈
from dotenv import load_dotenv

# 환경 변수 로드
load_dotenv()
base_dir = os.getenv('BASE_DIR')

In [2]:
data = np.concatenate([
    np.load(os.path.join(base_dir, f'seq_data/10words/seq_npy_1725553267_30_10.npy')),
    np.load(os.path.join(base_dir, f'seq_data/10words/seq_npy_flip_1725553267_30_10.npy')),
    np.load(os.path.join(base_dir, f'seq_data/10words/seq_npy_shift_1725553268_30_10.npy')),
    np.load(os.path.join(base_dir, f'seq_data/10words/seq_npy_flip_shift_1725553269_30_10.npy')),
], axis=0)

print(data.shape)
# (데이터의 개수, 프레임 사이즈, 한 프레임당 데이터 개수)

(7560, 30, 235)


In [3]:
from setting import actions
from keras.utils import to_categorical

# 데이터 분리 및 전처리
x_data = data[:, :, :-1]    # 시퀀스의 마지막 요소 제외한 모든 값 가져와 할당
labels = data[:, 0, -1]     # 마지막 요소는 레이블 값

# 원-핫 인코딩으로 변환
y_data = to_categorical(labels, num_classes=len(actions))

print(x_data.shape)
print(labels.shape)
print(y_data.shape)         # y_data 형태 -> [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...]
print(np.unique(labels))    # 레이블 값 출력


(7560, 30, 234)
(7560,)
(7560, 20)
[0. 1. 2. 3. 4. 5. 6. 7. 8. 9.]


In [4]:
from sklearn.model_selection import train_test_split

# 데이터셋 분할 (학습 데이터와 검증 데이터만 사용)
x_train, x_val, y_train, y_val = train_test_split(x_data, y_data, test_size=0.2, random_state=2024, stratify=labels)

print(x_train.shape, y_train.shape)
print(x_val.shape, y_val.shape)

(6048, 30, 234) (6048, 20)
(1512, 30, 234) (1512, 20)


In [5]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout

# 모델 정의
model = Sequential([
    LSTM(128, activation='tanh', input_shape=x_data.shape[1:3]),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(len(actions), activation='softmax'),
])

# 모델 컴파일 (최적화 알고리즘, 레이블 클래스 2개 이상일 때 사용하는 손실 함수, 모델평가지표)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 모델 요약
model.summary()



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 128)               185856    
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense (Dense)               (None, 64)                8256      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 20)                1300      
                                                                 
Total params: 195412 (763.33 KB)
Trainable params: 195412 (763.33 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [6]:
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping

# 모델 훈련
history = model.fit(
    x_train,
    y_train,
    validation_data=(x_val, y_val),
    epochs=100,
    batch_size=128,
    callbacks=[
        # save_best_only -> 모델 정확도가 이전보다 향상된 경우에만 모델 저장
        ModelCheckpoint('models/model.keras', monitor='val_accuracy', verbose=1, save_best_only=True, mode='auto'),
        # 정확도 개선이 없을시 학습률(factor) 0.5배로 감소, n 에포크 동안 개선 없을 경우 학습률 감소
        ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=20, verbose=1, mode='auto'),
        # early stopping 적용
        # EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True),
    ]
)

Epoch 1/100


47/48 [============================>.] - ETA: 0s - loss: 1.9505 - accuracy: 0.3329
Epoch 1: val_accuracy improved from -inf to 0.63228, saving model to models\model.keras
48/48 [==============================] - 7s 72ms/step - loss: 1.9460 - accuracy: 0.3347 - val_loss: 1.0782 - val_accuracy: 0.6323 - lr: 0.0010
Epoch 2/100
47/48 [============================>.] - ETA: 0s - loss: 1.0188 - accuracy: 0.6476
Epoch 2: val_accuracy improved from 0.63228 to 0.77315, saving model to models\model.keras
48/48 [==============================] - 3s 61ms/step - loss: 1.0170 - accuracy: 0.6481 - val_loss: 0.6379 - val_accuracy: 0.7731 - lr: 0.0010
Epoch 3/100
47/48 [============================>.] - ETA: 0s - loss: 0.6208 - accuracy: 0.7937
Epoch 3: val_accuracy improved from 0.77315 to 0.83862, saving model to models\model.keras
48/48 [==============================] - 3s 67ms/step - loss: 0.6202 - accuracy: 0.7941 - val_loss: 0.4624 - val_accuracy: 0.8386 - lr: 0.0010
Epoch 4/100
48

In [ ]:
import matplotlib.pyplot as plt

# 학습 데이터/검증 데이터 정확도/손실 그래프
fig, loss_ax = plt.subplots(figsize=(16, 10))
acc_ax = loss_ax.twinx()

loss_ax.plot(history.history['loss'], 'y', label='train loss')
loss_ax.plot(history.history['val_loss'], 'r', label='val loss')
loss_ax.set_xlabel('epoch')
loss_ax.set_ylabel('loss')
loss_ax.legend(loc='upper left')

acc_ax.plot(history.history['accuracy'], 'b', label='train accuracy')
acc_ax.plot(history.history['val_accuracy'], 'g', label='val accuracy')
acc_ax.set_ylabel('accuracy')
acc_ax.legend(loc='upper right')

plt.show()